# Set up example code

## Model setup

In [1]:
from __future__ import print_function

import subprocess

from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process import kernels
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import joblib

import matplotlib.cm as cm
import matplotlib.pyplot as plt

from IPython.display import display, clear_output


from scipy.linalg import lapack
from scipy import stats
import emcee
import numpy as np

import os
import pickle
from pathlib import Path

import src.reader as Reader

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Step 1: prepare input pickle file

### Load stuff from text files

In [2]:
# Read data files
RawData1M   = Reader.ReadData('input/MATTERTruncated/Data_PHENIX_AuAu200_RAACharged_0to10_2013.dat')
RawData2M   = Reader.ReadData('input/MATTERTruncated/Data_PHENIX_AuAu200_RAACharged_40to50_2013.dat')
RawData3M   = Reader.ReadData('input/MATTERTruncated/Data_ATLAS_PbPb2760_RAACharged_0to5_2015.dat')
RawData4M   = Reader.ReadData('input/MATTERTruncated/Data_ATLAS_PbPb2760_RAACharged_30to40_2015.dat')
RawData5M   = Reader.ReadData('input/MATTERTruncated/Data_CMS_PbPb5020_RAACharged_0to10_2017.dat')
RawData6M   = Reader.ReadData('input/MATTERTruncated/Data_CMS_PbPb5020_RAACharged_30to50_2017.dat')
RawData1L   = Reader.ReadData('input/LBTJake/Data_PHENIX_AuAu200_RAACharged_0to10_2013.dat')
RawData2L   = Reader.ReadData('input/LBTJake/Data_PHENIX_AuAu200_RAACharged_40to50_2013.dat')
RawData3L   = Reader.ReadData('input/LBTJake/Data_ATLAS_PbPb2760_RAACharged_0to5_2015.dat')
RawData4L   = Reader.ReadData('input/LBTJake/Data_ATLAS_PbPb2760_RAACharged_30to40_2015.dat')
RawData5L   = Reader.ReadData('input/LBTJake/Data_CMS_PbPb5020_RAACharged_0to10_2017.dat')
RawData6L   = Reader.ReadData('input/LBTJake/Data_CMS_PbPb5020_RAACharged_30to50_2017.dat')
RawData1ML1 = Reader.ReadData('input/MATTERLBT1/Data_PHENIX_AuAu200_RAACharged_0to10_2013.dat')
RawData2ML1 = Reader.ReadData('input/MATTERLBT1/Data_PHENIX_AuAu200_RAACharged_40to50_2013.dat')
RawData3ML1 = Reader.ReadData('input/MATTERLBT1/Data_ATLAS_PbPb2760_RAACharged_0to5_2015.dat')
RawData4ML1 = Reader.ReadData('input/MATTERLBT1/Data_ATLAS_PbPb2760_RAACharged_30to40_2015.dat')
RawData5ML1 = Reader.ReadData('input/MATTERLBT1/Data_CMS_PbPb5020_RAACharged_0to10_2017.dat')
RawData6ML1 = Reader.ReadData('input/MATTERLBT1/Data_CMS_PbPb5020_RAACharged_30to50_2017.dat')
RawData1ML2 = Reader.ReadData('input/MATTERLBT2/Data_PHENIX_AuAu200_RAACharged_0to10_2013.dat')
RawData2ML2 = Reader.ReadData('input/MATTERLBT2/Data_PHENIX_AuAu200_RAACharged_40to50_2013.dat')
RawData3ML2 = Reader.ReadData('input/MATTERLBT2/Data_ATLAS_PbPb2760_RAACharged_0to5_2015.dat')
RawData4ML2 = Reader.ReadData('input/MATTERLBT2/Data_ATLAS_PbPb2760_RAACharged_30to40_2015.dat')
RawData5ML2 = Reader.ReadData('input/MATTERLBT2/Data_CMS_PbPb5020_RAACharged_0to10_2017.dat')
RawData6ML2 = Reader.ReadData('input/MATTERLBT2/Data_CMS_PbPb5020_RAACharged_30to50_2017.dat')

# Read covariance
RawCov11L = Reader.ReadCovariance('input/LBT/Covariance_PHENIX_AuAu200_RAACharged_0to10_2013_PHENIX_AuAu200_RAACharged_0to10_2013_Jake.dat')
RawCov22L = Reader.ReadCovariance('input/LBT/Covariance_PHENIX_AuAu200_RAACharged_40to50_2013_PHENIX_AuAu200_RAACharged_40to50_2013_Jake.dat')
RawCov33L = Reader.ReadCovariance('input/LBT/Covariance_ATLAS_PbPb2760_RAACharged_0to5_2015_ATLAS_PbPb2760_RAACharged_0to5_2015_Jake.dat')
RawCov44L = Reader.ReadCovariance('input/LBT/Covariance_ATLAS_PbPb2760_RAACharged_30to40_2015_ATLAS_PbPb2760_RAACharged_30to40_2015_Jake.dat')
RawCov55L = Reader.ReadCovariance('input/LBT/Covariance_CMS_PbPb5020_RAACharged_0to10_2017_CMS_PbPb5020_RAACharged_0to10_2017_Jake.dat')
RawCov66L = Reader.ReadCovariance('input/LBT/Covariance_CMS_PbPb5020_RAACharged_30to50_2017_CMS_PbPb5020_RAACharged_30to50_2017_Jake.dat')

RawCov11E = Reader.ReadCovariance('input/Example/Covariance_PHENIX_AuAu200_RAACharged_0to10_2013_PHENIX_AuAu200_RAACharged_0to10_2013_SmallL.dat')
RawCov22E = Reader.ReadCovariance('input/Example/Covariance_PHENIX_AuAu200_RAACharged_40to50_2013_PHENIX_AuAu200_RAACharged_40to50_2013_SmallL.dat')
RawCov33E = Reader.ReadCovariance('input/Example/Covariance_ATLAS_PbPb2760_RAACharged_0to5_2015_ATLAS_PbPb2760_RAACharged_0to5_2015_SmallL.dat')
RawCov44E = Reader.ReadCovariance('input/Example/Covariance_ATLAS_PbPb2760_RAACharged_30to40_2015_ATLAS_PbPb2760_RAACharged_30to40_2015_SmallL.dat')
RawCov55E = Reader.ReadCovariance('input/Example/Covariance_CMS_PbPb5020_RAACharged_0to10_2017_CMS_PbPb5020_RAACharged_0to10_2017_SmallL.dat')
RawCov66E = Reader.ReadCovariance('input/Example/Covariance_CMS_PbPb5020_RAACharged_30to50_2017_CMS_PbPb5020_RAACharged_30to50_2017_SmallL.dat')

# Read design points
RawDesignM   = Reader.ReadDesign('input/MATTERTruncated/Design.dat')
RawDesignL   = Reader.ReadDesign('input/LBTJake/Design.dat')
RawDesignML1 = Reader.ReadDesign('input/MATTERLBT1/Design.dat')
RawDesignML2 = Reader.ReadDesign('input/MATTERLBT2/Design.dat')

# Read model prediction
RawPrediction1M   = Reader.ReadPrediction('input/MATTERTruncated/Prediction_PHENIX_AuAu200_RAACharged_0to10_2013.dat')
RawPrediction2M   = Reader.ReadPrediction('input/MATTERTruncated/Prediction_PHENIX_AuAu200_RAACharged_40to50_2013.dat')
RawPrediction3M   = Reader.ReadPrediction('input/MATTERTruncated/Prediction_ATLAS_PbPb2760_RAACharged_0to5_2015.dat')
RawPrediction4M   = Reader.ReadPrediction('input/MATTERTruncated/Prediction_ATLAS_PbPb2760_RAACharged_30to40_2015.dat')
RawPrediction5M   = Reader.ReadPrediction('input/MATTERTruncated/Prediction_CMS_PbPb5020_RAACharged_0to10_2017.dat')
RawPrediction6M   = Reader.ReadPrediction('input/MATTERTruncated/Prediction_CMS_PbPb5020_RAACharged_30to50_2017.dat')
RawPrediction1L   = Reader.ReadPrediction('input/LBTJake/Prediction_PHENIX_AuAu200_RAACharged_0to10_2013.dat')
RawPrediction2L   = Reader.ReadPrediction('input/LBTJake/Prediction_PHENIX_AuAu200_RAACharged_40to50_2013.dat')
RawPrediction3L   = Reader.ReadPrediction('input/LBTJake/Prediction_ATLAS_PbPb2760_RAACharged_0to5_2015.dat')
RawPrediction4L   = Reader.ReadPrediction('input/LBTJake/Prediction_ATLAS_PbPb2760_RAACharged_30to40_2015.dat')
RawPrediction5L   = Reader.ReadPrediction('input/LBTJake/Prediction_CMS_PbPb5020_RAACharged_0to10_2017.dat')
RawPrediction6L   = Reader.ReadPrediction('input/LBTJake/Prediction_CMS_PbPb5020_RAACharged_30to50_2017.dat')
RawPrediction1ML1 = Reader.ReadPrediction('input/MATTERLBT1/Prediction_PHENIX_AuAu200_RAACharged_0to10_2013.dat')
RawPrediction2ML1 = Reader.ReadPrediction('input/MATTERLBT1/Prediction_PHENIX_AuAu200_RAACharged_40to50_2013.dat')
RawPrediction3ML1 = Reader.ReadPrediction('input/MATTERLBT1/Prediction_ATLAS_PbPb2760_RAACharged_0to5_2015.dat')
RawPrediction4ML1 = Reader.ReadPrediction('input/MATTERLBT1/Prediction_ATLAS_PbPb2760_RAACharged_30to40_2015.dat')
RawPrediction5ML1 = Reader.ReadPrediction('input/MATTERLBT1/Prediction_CMS_PbPb5020_RAACharged_0to10_2017.dat')
RawPrediction6ML1 = Reader.ReadPrediction('input/MATTERLBT1/Prediction_CMS_PbPb5020_RAACharged_30to50_2017.dat')
RawPrediction1ML2 = Reader.ReadPrediction('input/MATTERLBT2/Prediction_PHENIX_AuAu200_RAACharged_0to10_2013.dat')
RawPrediction2ML2 = Reader.ReadPrediction('input/MATTERLBT2/Prediction_PHENIX_AuAu200_RAACharged_40to50_2013.dat')
RawPrediction3ML2 = Reader.ReadPrediction('input/MATTERLBT2/Prediction_ATLAS_PbPb2760_RAACharged_0to5_2015.dat')
RawPrediction4ML2 = Reader.ReadPrediction('input/MATTERLBT2/Prediction_ATLAS_PbPb2760_RAACharged_30to40_2015.dat')
RawPrediction5ML2 = Reader.ReadPrediction('input/MATTERLBT2/Prediction_CMS_PbPb5020_RAACharged_0to10_2017.dat')
RawPrediction6ML2 = Reader.ReadPrediction('input/MATTERLBT2/Prediction_CMS_PbPb5020_RAACharged_30to50_2017.dat')

### Run this block for RHIC + LHC (LBT)

In [3]:
# Initialize empty dictionary
AllData = {}

# Basic information
AllData["systems"] = ["AuAu200", "PbPb2760", "PbPb5020"]
AllData["keys"] = RawDesignL["Parameter"]
AllData["labels"] = RawDesignL["Parameter"]
AllData["ranges"] = [(0.01, 2), (0.01, 20), (0.01, 2), (0.01, 20)]
# AllData["ranges"] = [(0.01, 2), (0.01, 10), (0.01, 2), (0.01, 10)]
AllData["observables"] = [('R_AA', ['C0', 'C1'])]

# Data points
Data = {"AuAu200": {"R_AA": {"C0": RawData1L["Data"], "C1": RawData2L["Data"]}},
       "PbPb2760": {"R_AA": {"C0": RawData3L["Data"], "C1": RawData4L["Data"]}},
       "PbPb5020": {"R_AA": {"C0": RawData5L["Data"], "C1": RawData6L["Data"]}}}

# Model predictions
Prediction = {"AuAu200": {"R_AA": {"C0": {"Y": RawPrediction1L["Prediction"], "x": RawData1L["Data"]['x']},
                                   "C1": {"Y": RawPrediction2L["Prediction"], "x": RawData2L["Data"]['x']}}},
             "PbPb2760": {"R_AA": {"C0": {"Y": RawPrediction3L["Prediction"], "x": RawData3L["Data"]['x']},
                                   "C1": {"Y": RawPrediction4L["Prediction"], "x": RawData4L["Data"]['x']}}},
             "PbPb5020": {"R_AA": {"C0": {"Y": RawPrediction5L["Prediction"], "x": RawData5L["Data"]['x']},
                                   "C1": {"Y": RawPrediction6L["Prediction"], "x": RawData6L["Data"]['x']}}}}

# Covariance matrices - the indices are [system][measurement1][measurement2], each one is a block of matrix
Covariance = Reader.InitializeCovariance(Data)
Covariance["AuAu200"][("R_AA", "C0")][("R_AA", "C0")]  = Reader.EstimateCovariance(RawData1L, RawData1L, SysLength = {"default": 0.2})
Covariance["AuAu200"][("R_AA", "C1")][("R_AA", "C1")]  = Reader.EstimateCovariance(RawData2L, RawData2L, SysLength = {"default": 0.2})
Covariance["PbPb2760"][("R_AA", "C0")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData3L, RawData3L, SysLength = {"default": 0.2})
Covariance["PbPb2760"][("R_AA", "C1")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData4L, RawData4L, SysLength = {"default": 0.2})
Covariance["PbPb5020"][("R_AA", "C0")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData5L, RawData5L, SysLength = {"default": 0.2})
Covariance["PbPb5020"][("R_AA", "C1")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData6L, RawData6L, SysLength = {"default": 0.2})

# This is how we can add off-diagonal matrices
# Covariance["PbPb5020"][("R_AA", "C0")][("R_AA", "C1")] = Reader.EstimateCovariance(RawData5, RawData6, SysLength = {"default": 100}, SysStrength = {"default": 0.1})
# Covariance["PbPb5020"][("R_AA", "C1")][("R_AA", "C0")] = Reader.EstimateCovariance(RawData6, RawData5, SysLength = {"default": 100}, SysStrength = {"default": 0.1})

# This is how we can supply external pre-generated matrices
# Covariance["AuAu200"][("R_AA", "C0")][("R_AA", "C0")] = RawCov11L["Matrix"]
# Covariance["AuAu200"][("R_AA", "C1")][("R_AA", "C1")] = RawCov22L["Matrix"]
# Covariance["PbPb2760"][("R_AA", "C0")][("R_AA", "C0")] = RawCov33L["Matrix"]
# Covariance["PbPb2760"][("R_AA", "C1")][("R_AA", "C1")] = RawCov44L["Matrix"]
# Covariance["PbPb5020"][("R_AA", "C0")][("R_AA", "C0")] = RawCov55L["Matrix"]
# Covariance["PbPb5020"][("R_AA", "C1")][("R_AA", "C1")] = RawCov66L["Matrix"]

# Assign data to the dictionary
AllData["design"] = RawDesignL["Design"]
AllData["model"] = Prediction
AllData["data"] = Data
AllData["cov"] = Covariance

# Save to the desired pickle file
with open('input/default.p', 'wb') as handle:
    pickle.dump(AllData, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [4]:
from src import lazydict, emulator
EmulatorAuAu200 = emulator.Emulator.from_cache('AuAu200')
EmulatorPbPb2760 = emulator.Emulator.from_cache('PbPb2760')
EmulatorPbPb5020 = emulator.Emulator.from_cache('PbPb5020')

In [5]:
import src
src.Initialize()
from src import mcmc
chain = mcmc.Chain()
MCMCSamples = chain.load()

### K-nearest neighbor test

Distance is defined with Gaussian with some distance measure.  Let's put it as 0.1 to start with.  Then we simply loop over all model calculations and calculate the weighted average for given parameter set.

In [7]:
%config InlineBackend.close_figures = True

SystemCount = len(AllData["systems"])
artist1 = [[None, None], [None, None], [None, None]]
artist2 = [[None, None], [None, None], [None, None]]

figure, axes = plt.subplots(figsize = (10, 3 * SystemCount), ncols = 2, nrows = SystemCount)

# Plot data points
for s1 in range(0, SystemCount):
    for s2 in range(0, 2):
        axes[s1][s2].set_xlabel(r"$p_{T}$")
        axes[s1][s2].set_ylabel(r"$R_{AA}$")
        
        S1 = AllData["systems"][s1]
        O  = AllData["observables"][0][0]
        S2 = AllData["observables"][0][1][s2]
        
        DX = AllData["data"][S1][O][S2]['x']
        DY = AllData["data"][S1][O][S2]['y']
        DE = np.sqrt(AllData["data"][S1][O][S2]['yerr']['stat'][:,0]**2 + AllData["data"][S1][O][S2]['yerr']['sys'][:,0]**2)
        
        axes[s1][s2].errorbar(range(1, len(DY)+1), DY, yerr = DE, fmt='ro', label="Measurements")

plt.close(figure)

# Update prediction from GP and from nearest neighbor
def RunPrediction(A, B, C, D):
    # print ("Parameters = ", A, B, C, D)
    Examples = [[A, B, C, D]]

    figure.suptitle('Parameters = ({:.3f}, {:.2f}, {:.3f}, {:.2f})'.format(A, B, C, D), fontsize=16)
    
    N = 100
    TempPrediction = {"AuAu200": EmulatorAuAu200.predict(Examples),
                 "PbPb2760": EmulatorPbPb2760.predict(Examples),
                 "PbPb5020": EmulatorPbPb5020.predict(Examples)}
    
    NewPrediction = {"AuAu200": {"R_AA": {"C0": np.zeros(AllData["model"]["AuAu200"]["R_AA"]["C0"]["Y"].shape[-1]),
                                          "C1": np.zeros(AllData["model"]["AuAu200"]["R_AA"]["C1"]["Y"].shape[-1])}},
        "PbPb2760": {"R_AA": {"C0": np.zeros(AllData["model"]["PbPb2760"]["R_AA"]["C0"]["Y"].shape[-1]),
                              "C1": np.zeros(AllData["model"]["PbPb2760"]["R_AA"]["C1"]["Y"].shape[-1])}},
        "PbPb5020": {"R_AA": {"C0": np.zeros(AllData["model"]["PbPb5020"]["R_AA"]["C0"]["Y"].shape[-1]),
                              "C1": np.zeros(AllData["model"]["PbPb5020"]["R_AA"]["C1"]["Y"].shape[-1])}}}
    
    TotalWeight = 0
    Sigma = 0.05
    for iP in range(AllData["design"].shape[0]):   # loop over design points
        Displacement = AllData["design"][iP] - Examples[0]
        for i in range(0, 4):
            Displacement[i] = Displacement[i] / (AllData["ranges"][i][1] - AllData["ranges"][i][0])
        Distance = np.linalg.norm(Displacement)
        Weight = np.exp(-(Distance * Distance) / (2 * Sigma * Sigma))
        TotalWeight = TotalWeight + Weight
        NewPrediction["AuAu200"]["R_AA"]["C0"] += AllData["model"]["AuAu200"]["R_AA"]["C0"]["Y"][iP] * Weight;
        NewPrediction["AuAu200"]["R_AA"]["C1"] += AllData["model"]["AuAu200"]["R_AA"]["C1"]["Y"][iP] * Weight;
        NewPrediction["PbPb2760"]["R_AA"]["C0"] += AllData["model"]["PbPb2760"]["R_AA"]["C0"]["Y"][iP] * Weight;
        NewPrediction["PbPb2760"]["R_AA"]["C1"] += AllData["model"]["PbPb2760"]["R_AA"]["C1"]["Y"][iP] * Weight;
        NewPrediction["PbPb5020"]["R_AA"]["C0"] += AllData["model"]["PbPb5020"]["R_AA"]["C0"]["Y"][iP] * Weight;
        NewPrediction["PbPb5020"]["R_AA"]["C1"] += AllData["model"]["PbPb5020"]["R_AA"]["C1"]["Y"][iP] * Weight;
    NewPrediction["AuAu200"]["R_AA"]["C0"] /= TotalWeight
    NewPrediction["AuAu200"]["R_AA"]["C1"] /= TotalWeight
    NewPrediction["PbPb2760"]["R_AA"]["C0"] /= TotalWeight
    NewPrediction["PbPb2760"]["R_AA"]["C1"] /= TotalWeight
    NewPrediction["PbPb5020"]["R_AA"]["C0"] /= TotalWeight
    NewPrediction["PbPb5020"]["R_AA"]["C1"] /= TotalWeight

    for s1 in range(0, SystemCount):
        for s2 in range(0, 2):
            S1 = AllData["systems"][s1]
            O  = AllData["observables"][0][0]
            S2 = AllData["observables"][0][1][s2]
            
            DX = AllData["data"][S1][O][S2]['x']
            DY = AllData["data"][S1][O][S2]['y']
            DE = np.sqrt(AllData["data"][S1][O][S2]['yerr']['stat'][:,0]**2 + AllData["data"][S1][O][S2]['yerr']['sys'][:,0]**2)

            for i, y in enumerate(TempPrediction[S1][O][S2]):
                if artist1[s1][s2] != None:
                    artist1[s1][s2].remove()
                artist1[s1][s2], = axes[s1][s2].plot(range(1, len(y)+1), y, 'b-', alpha=1, label="Posterior" if i==0 else '')
            
            y2 = NewPrediction[S1][O][S2]
            
            if artist2[s1][s2] != None:
                artist2[s1][s2].remove()
            artist2[s1][s2], = axes[s1][s2].plot(range(1, len(y2)+1), y2, 'g-', alpha=1)
            
    display(figure)
    print("Parameters = ", A, B, C, D)
    print("Likelihood = ", chain.log_posterior([A, B, C, D]))


Default = [0.176, 3.52, 0.290, 4.56] # Paper LBT median
# Default = [0.052, 17, 0.4, 1.5]
# Default = [0.02, 19.95, 0.35, 0.02] # LBT best fit
# Default = [1.06, 0.541, 0.568, 2.488] # LBT design 1
# Default = [0.329, 2.909, 0.208, 4.161] # LBT design 2
# Default = [0.983, 1.656, 0.293, 0.933] # LBT design 3
# Default = [0.02, 0.04, 0.260, 0.18] # Test point 1 LBT
# Default = [0.02, 0.04, 1.666, 0.18] # Test point 2 LBT

interactive_plot = interactive(RunPrediction,
    A = widgets.FloatSlider(min = AllData["ranges"][0][0], max = AllData["ranges"][0][1], step = 0.001, continuous_update = False, value = Default[0]),
    B = widgets.FloatSlider(min = AllData["ranges"][1][0], max = AllData["ranges"][1][1], step = 0.010, continuous_update = False, value = Default[1]),
    C = widgets.FloatSlider(min = AllData["ranges"][2][0], max = AllData["ranges"][2][1], step = 0.001, continuous_update = False, value = Default[2]),
    D = widgets.FloatSlider(min = AllData["ranges"][3][0], max = AllData["ranges"][3][1], step = 0.010, continuous_update = False, value = Default[3]))

output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot

# print(interactive_plot.children[-1].layout)

interactive(children=(FloatSlider(value=0.176, continuous_update=False, description='A', max=2.0, min=0.01, st…